# Importing Data and reading Xlsx File

Main Objectives

- 1. **Predicting Incurred Amount (Regression)**
   
Estimate the monetary cost of a claim — the total liability the insurer expects to pay (includes medical bills, repairs, settlements, etc.).

Why it matters to the business:

- Accurate Reserving

- Insurers must set aside money (reserves) to cover expected payouts. Overestimating ties up cash unnecessarily; underestimating causes solvency risks.

- Financial Forecasting

- Enables CFOs and actuaries to project financial exposure, capital needs, and reinsurance planning.

- Fraud & Anomaly Detection

- Large differences between predicted vs. actual incurred amounts can flag outlier claims or suspicious behavior.

- Operational Planning

H- elps managers allocate experienced adjusters and legal resources to high-value claims early.

- 2. **Classifying High-Cost Claims (Binary Classification: 0/1)**

Identify whether a claim will become "high-cost" (e.g., in top 1–5% of cost distribution) as early as possible.

Why it matters to the business:

- Proactive Intervention

- Early detection of costly claims allows escalation to senior adjusters, use of special investigators, or negotiation strategies.

- Cost Containment

- Timely action can prevent claims from ballooning (e.g., managing medical treatments, dispute resolution, rental periods).

- Customer Strategy

- Flagging high-cost claims helps tailor communication, retain key customers, or make legal preparations early.

- Portfolio Risk Profiling

- Helps the underwriting team understand which customer or product segments produce high-cost claims and adjust policies or pricing.

In [ ]:
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

data_file = pd.ExcelFile('Data_Scientist_Interview_Task.xlsx')

print(data_file.sheet_names)

In [ ]:
df = pd.read_excel(data_file, sheet_name='Data')
df

# Data Cleansing and Issue Identification

In [ ]:
df.info()
df.isnull().sum()

* weather conditions is the only feature which has null values which need to be transformed

In [ ]:
df['Weather_conditions'].unique()

In [ ]:
df.describe()

In [ ]:
df['date_of_loss'] = pd.to_datetime(df['date_of_loss'], errors='coerce')
df

# Fill Nan values

In [ ]:
plt.figure(figsize=(14, 6))
sns.heatmap(df.isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title("Missing Values Heatmap")
plt.show()

In [ ]:
cat_cols = df.select_dtypes(include='object').columns
df[cat_cols] = df[cat_cols].fillna('Missing')
df

In [ ]:
num_cols = df.select_dtypes(include=np.number).columns

df[num_cols] = df[num_cols].fillna(0)
df

In [ ]:
df.isnull().sum()

In [ ]:
df['TP_injury_whiplash'].unique()

# Outlier Detection

In [ ]:
important_numerical_cols = [
    'Incurred', 'Capped Incurred', 'Notification_period', 'Inception_to_loss',
    'TP_type_driver', 'TP_type_pass_front', 'TP_type_pass_back',
    'TP_type_pedestrian', 'TP_type_other',
    'TP_injury_whiplash', 'TP_injury_traumatic', 'TP_injury_fatality'
]

def detect_outliers(df, columns):
    outlier_summary = {}
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)]
        outlier_count = outliers.shape[0]
        outlier_pct = (outlier_count / df.shape[0]) * 100
        outlier_summary[col] = {
            "Lower Bound": lower,
            "Upper Bound": upper,
            "Outlier Count": outlier_count,
            "Outlier %": round(outlier_pct, 2)
        }

        plt.figure(figsize=(10, 1.5))
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot of {col}')
        plt.show()

    return pd.DataFrame(outlier_summary).T

outlier_report = detect_outliers(df, important_numerical_cols)

outlier_report.sort_values("Outlier %", ascending=False)


In [ ]:
# Define threshold for top 1% of Incurred
threshold_99 = df['Incurred'].quantile(0.99)

# Filter top 1% claims
outliers = df[df['Incurred'] >= threshold_99]

plt.figure(figsize=(10, 6))
sns.histplot(outliers['Incurred'], bins=30, color='crimson', kde=True)
plt.title("Top 1% Most Expensive Claims (Incurred ≥ {:.2f})".format(threshold_99))
plt.xlabel("Incurred Amount")
plt.ylabel("Count")
plt.show()

print(f"Top 1% threshold: {threshold_99:.2f}")
print(f"Number of outliers: {len(outliers)}")


#### Some explanation of the outlier results:

* TP_injury_whiplash, 1438 claims (~18.7%) have this injury.
* Outlier counts in the Incurred column (979 claims or 12.73%) indicate a significant number of claims lie outside expected normal ranges — these are candidates for further investigation or treatment in modeling (e.g., capping, transforming, or excluding). The negative values, despite showing financial gain for the insurance company, they could be even fraudulent transactions which need to be identified further.
* Variables with very low outlier thershold: These are categorical or numeric variables with mostly expected values, and only a few unusual ones that could be data entry errors or very rare claim types.
* Inception to loss has zero outliers: This means all values fall within the expected bounds. No outliers detected here.
* **Note:** Outliers are important indicators which cannot be excluded from the dataframe, else we will miss important information. 

## EDA to better understand the data

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 5))
sns.histplot(df['Notification_period'], bins=30, kde=True)
plt.title("Distribution of Notification Period")
plt.xlabel("Notification Period (days)")
plt.show()

plt.figure(figsize=(12, 5))
sns.histplot(df['Incurred'], bins=50, color='blue', label='Incurred', alpha=0.6)
sns.histplot(df['Capped Incurred'], bins=50, color='orange', label='Capped Incurred', alpha=0.6)
plt.legend()
plt.title("Distribution of Incurred vs Capped Incurred")
plt.xlabel("Claim Amount")
plt.show()



## Statistical Comparison between capped incured and whiplash injuries 

In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu, kruskal

whiplash_yes = df[df['TP_injury_whiplash'] == 1]['Incurred']
whiplash_no = df[df['TP_injury_whiplash'] == 0]['Incurred']

print("Mean Incurred Amount (Whiplash):", whiplash_yes.mean())
print("Mean Incurred Amount (No Whiplash):", whiplash_no.mean())
print("Median Incurred Amount (Whiplash):", whiplash_yes.median())
print("Median Incurred Amount (No Whiplash):", whiplash_no.median())
print("Sample Sizes:", len(whiplash_yes), "vs", len(whiplash_no))

grouped = [df[df['TP_injury_whiplash'] == cat]['Incurred'] for cat in sorted(df['TP_injury_whiplash'].unique())]

stat, p = kruskal(*grouped)
print(f"Kruskal–Wallis H-statistic = {stat:.2f}, p-value = {p:.4f}")



**Explanation:** We statistically compared claims with and without whiplash injuries using Kruskal statistical tests (comparing continuous variable with multi-class variable). The test confirmed that claims involving whiplash have significantly higher incurred costs than those without.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='TP_injury_whiplash', y='Incurred', data=df)
plt.title("Incurred Amount by Whiplash Injury Class")
plt.xlabel("Whiplash Injury Code")
plt.ylabel("Incurred Amount")
plt.ylim(0, df['Incurred'].quantile(0.95))
plt.show()

## Claim frequency by hour of day

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(x='Time_hour', data=df, palette='viridis')
plt.title("Number of Claims by Hour of Day")
plt.xlabel("Hour (24-hour format)")
plt.ylabel("Number of Claims")
plt.xticks(range(0, 24))
plt.show()

## Claims per region

In [ ]:
tp_region_cols = [col for col in df.columns if col.startswith('TP_region_')]
region_counts = df[tp_region_cols].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=region_counts.index, y=region_counts.values, palette="coolwarm")
plt.xticks(rotation=45)
plt.title("Claims Count by Region")
plt.ylabel("Number of Claims")
plt.xlabel("Region")
plt.show()

## Distribution of Incurred cost (Log scale)

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(np.log1p(df['Incurred']), bins=50, kde=True, color='green')
plt.title("Log-Scaled Distribution of Incurred Claim Amount")
plt.xlabel("log(Incurred + 1)")
plt.show()

**Insight:** Most claims are low-to-moderate cost; few are very large (heavy tail).

## Incurred by Third party type

In [ ]:
tp_types = ['TP_type_driver', 'TP_type_pedestrian', 'TP_type_cyclist', 'TP_type_bike']
melted = df.melt(id_vars='Incurred', value_vars=tp_types, var_name='TP_Type', value_name='Present')
melted = melted[melted['Present'] == 1]

plt.figure(figsize=(10, 6))
sns.boxplot(x='TP_Type', y='Incurred', data=melted)
plt.ylim(0, df['Incurred'].quantile(0.95))
plt.title("Claim Cost by Third Party Type")
plt.xticks(rotation=20)
plt.show()

## Notification delay vs cost

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Notification_period', y='Incurred', data=df, alpha=0.3)
plt.ylim(0, df['Incurred'].quantile(0.95))
plt.title("Notification Delay vs. Incurred Claim Amount")
plt.xlabel("Days Between Incident and Notification")
plt.ylabel("Incurred")
plt.show()

**Note:** Majority of incidents have been incurred within the first 200 days from the notification and incident

## Proportion of Injuries

In [ ]:
injury_cols = [col for col in df.columns if col.startswith("TP_injury_")]
injury_counts = df[injury_cols].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 5))
sns.barplot(x=injury_counts.index, y=injury_counts.values, palette='rocket')
plt.title("Frequency of Injury Types")
plt.xticks(rotation=45)
plt.ylabel("Number of Claims")
plt.show()

## Notification Delay vs Incurred by Fault

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Notification_period', y='Incurred',
                hue='PH_considered_TP_at_fault', alpha=0.5, palette='coolwarm')
plt.ylim(0, df['Incurred'].quantile(0.95))
plt.title("Incurred vs. Notification Delay (Colored by At Fault)")
plt.xlabel("Notification Period (days)")
plt.ylabel("Incurred")
plt.legend(title="TP At Fault (1=True)")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

df['year'] = df['date_of_loss'].dt.year
df['month'] = df['date_of_loss'].dt.month
df['day'] = df['date_of_loss'].dt.day
df['weekday'] = df['date_of_loss'].dt.day_name()

monthly_counts = df.groupby(pd.Grouper(key='date_of_loss', freq='M')).size()

monthly_counts.plot(figsize=(12,6), title='Number of Claims per Month')
plt.xlabel('Month')
plt.ylabel('Number of Claims')
plt.show()


In [ ]:
yearly_counts = df.groupby(pd.Grouper(key='date_of_loss', freq='Y')).size()

yearly_counts.plot(figsize=(12,6), title='Number of Claims per Year')
plt.xlabel('Month')
plt.ylabel('Number of Claims')
plt.show()

# Feature Engineering

In [ ]:
df['Late_Report'] = df['Notification_period'] > 7

urban_regions = ['TP_region_london', 'TP_region_outerldn', 'TP_region_westmid']
df['Urban_Region'] = df[urban_regions].sum(axis=1) > 0

threshold_95 = df['Incurred'].quantile(0.95)
df['High_Cost_Claim'] = df['Incurred'] >= threshold_95

df


In [ ]:
sns.countplot(x='Late_Report', hue='High_Cost_Claim', data=df)
plt.title("High Cost Claims by Late Reporting Status")
plt.xlabel("Late Report")
plt.ylabel("Count")
plt.show()

sns.boxplot(x='Urban_Region', y='Incurred', data=df)
plt.title("Incurred Amounts in Urban vs Non-Urban Regions")
plt.ylim(0, df['Incurred'].quantile(0.95))
plt.show()

In [ ]:
df['Time_Bucket'] = pd.cut(df['Time_hour'],
                           bins=[-1, 6, 12, 17, 21, 24],
                           labels=['Night', 'Morning', 'Afternoon', 'Evening', 'Late night'])

df['Notification_Delay_Bucket'] = pd.cut(df['Notification_period'],
                                         bins=[-10, 0, 3, 7, 100],
                                         labels=['Early', 'Fast', 'Late', 'Very Late'])
df['Multiple_Injuries'] = df[[col for col in df.columns if col.startswith("TP_injury_")]].sum(axis=1) > 1
df['TP_Type_Count'] = df[[col for col in df.columns if col.startswith("TP_type_")]].sum(axis=1)
df['Log_Incurred'] = np.log1p(df['Incurred'])
df['Weekend_Incident'] = pd.to_datetime(df['date_of_loss']).dt.dayofweek >= 5
df

In [ ]:
numerical_cols = df.select_dtypes(include=['number']).columns.tolist()
numerical_cols

In [ ]:
## Claim Seasonality Heatmap

In [ ]:
df['month'] = df['date_of_loss'].dt.month
df['weekday'] = df['date_of_loss'].dt.day_name()

pivot = df.pivot_table(index='weekday', columns='month', values='Incurred', aggfunc='count')
sns.heatmap(pivot, cmap='YlGnBu')

monthly_stats = df.groupby('month').agg({
    'Incurred': ['sum', 'mean', 'count']
}).reset_index()

monthly_stats.columns = ['Month', 'Total_Cost', 'Average_Cost', 'Claim_Count']

plt.figure(figsize=(12,6))
plt.plot(monthly_stats['Month'], monthly_stats['Total_Cost'], marker='o', label='Total Incurred')
plt.title('Total Claim Cost Over Time')
plt.xlabel('Month')
plt.ylabel('Total Incurred')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,6))
plt.plot(monthly_stats['Month'], monthly_stats['Average_Cost'], marker='o', color='orange', label='Average Incurred')
plt.title('Average Claim Cost Over Time')
plt.xlabel('Month')
plt.ylabel('Average Incurred')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,6))
plt.bar(monthly_stats['Month'], monthly_stats['Claim_Count'], color='gray')
plt.title('Number of Claims per Month')
plt.xlabel('Month')
plt.ylabel('Claim Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Label Encoding with one hot encoding

**One-Hot Encoding treats categories as independent and unordered, preventing the model from assuming any hierarchy.**

In [ ]:
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

df = pd.get_dummies(df, columns=categorical_cols, drop_first=False)
df

In [ ]:
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)
df

In [ ]:
df.columns

## Correlation Heatmap (helps in removing multicolinearity)

In [ ]:
chunk_size = 10

cols = df.columns

num_chunks = 5

for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = start_idx + chunk_size
    cols_subset = cols[start_idx:end_idx]
    
    corr = df[cols_subset].corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
    plt.title(f'Correlation Heatmap: Columns {start_idx} to {end_idx-1}')
    plt.tight_layout()
    plt.show()

## Outlier Handling with Robust Scaler

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

num_cols = ['Notification_period',
             'Inception_to_loss',
             'Time_hour',
             'Vechile_registration_present',
             'Incident_details_present',
             'Injury_details_present',
             'TP_type_insd_pass_back',
             'TP_type_insd_pass_front',
             'TP_type_driver',
             'TP_type_pass_back',
             'TP_type_pass_front',
             'TP_type_bike',
             'TP_type_cyclist',
             'TP_type_pass_multi',
             'TP_type_pedestrian',
             'TP_type_other',
             'TP_type_nk',
             'TP_injury_whiplash',
             'TP_injury_traumatic',
             'TP_injury_fatality',
             'TP_injury_unclear',
             'TP_injury_nk',
             'TP_region_eastang',
             'TP_region_eastmid',
             'TP_region_london',
             'TP_region_north',
             'TP_region_northw',
             'TP_region_outerldn',
             'TP_region_scotland',
             'TP_region_southe',
             'TP_region_southw',
             'TP_region_wales',
             'TP_region_westmid',
             'TP_region_yorkshire',
             'TP_Type_Count']

df[num_cols] = scaler.fit_transform(df[num_cols])

df.columns

## Drop multi-collinear features

In [ ]:
def remove_multicollinear_features(df, threshold=0.6):
    
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
    
    print(f"Columns to drop due to multicollinearity (corr > {threshold}): {to_drop}")
    
    return df.drop(columns=to_drop)

df_clean = remove_multicollinear_features(df[num_cols], threshold=0.6)
df_clean

In [ ]:
df_clean.columns

In [ ]:
df.columns

In [ ]:
cols_to_add = [col for col in df.columns if col not in df_clean.columns]

df_extra = df[cols_to_add]

df_combined = pd.concat([df_clean.reset_index(drop=True), df_extra.reset_index(drop=True)], axis=1)
df_combined

In [ ]:
df.columns

In [ ]:
df['Incurred'].unique()

In [ ]:
df.groupby(['Weather_conditions_NORMAL', 'Urban_Region'])['Incurred'].mean().unstack().plot(kind='bar')
df

In [ ]:
df.groupby(['Weather_conditions_SNOW,ICE,FOG', 'Urban_Region'])['Incurred'].mean().unstack().plot(kind='bar')
df

# Target Variable Analysis

In [ ]:
num_zeros = (df['Incurred'] == 0).sum()
percent_zeros = 100 * num_zeros / len(df)

print(f"Number of zero Incurred values: {num_zeros}")
print(f"Percentage of zeros: {percent_zeros:.2f}%")


In [ ]:
df.columns

In [ ]:
df['Incurred'].unique()

# 1. Regression Analysis (Machine Learning and best model selection)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd


X = df.drop(columns=['Incurred', 'Capped Incurred', 'Log_Incurred', 'date_of_loss'])
y = np.log1p(df['Incurred'])
y = y.fillna(0)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

models = {
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42)
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    results[name] = {
        'MSE': mean_squared_error(y_test, y_pred),
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R2': r2_score(y_test, y_pred)
    }

results_df = pd.DataFrame(results).T.sort_values(by='RMSE')
print(results_df)



In [ ]:
model = GradientBoostingRegressor().fit(X_train, y_train)
importances = model.feature_importances_
feat_importance = pd.Series(importances, index=X.columns).sort_values(ascending=False)

feat_importance.head(20).plot(kind='barh', figsize=(10, 8), title="Top Feature Importances")
plt.gca().invert_yaxis()
plt.show()

## Retrain model Gradient Boosting with top 10, 20, 25, 30 features

In [ ]:
top_features = feat_importance.head(10).index

X_train_top = X_train[top_features]
X_test_top = X_test[top_features]

model_top = GradientBoostingRegressor(random_state=42)
model_top.fit(X_train_top, y_train)

y_pred_top = model_top.predict(X_test_top)

mae = mean_absolute_error(y_test, y_pred_top)
mse = mean_squared_error(y_test, y_pred_top)
r2 = r2_score(y_test, y_pred_top)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Top-10 Features Model Performance:")
print(f"MAE:  {mae:.4f}")
print(f"MSE:  {mse:.4f}")
print(f"R2:   {r2:.4f}")
print(f"RMSE: {rmse: .4f}")

In [ ]:
top_features = feat_importance.head(20).index

X_train_top = X_train[top_features]
X_test_top = X_test[top_features]

model_top = GradientBoostingRegressor(random_state=42)
model_top.fit(X_train_top, y_train)

y_pred_top = model_top.predict(X_test_top)

mae = mean_absolute_error(y_test, y_pred_top)
mse = mean_squared_error(y_test, y_pred_top)
r2 = r2_score(y_test, y_pred_top)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Top-20 Features Model Performance:")
print(f"MAE:  {mae:.4f}")
print(f"MSE:  {mse:.4f}")
print(f"R2:   {r2:.4f}")
print(f"RMSE: {rmse: .4f}")

In [ ]:
top_features = feat_importance.head(25).index

X_train_top = X_train[top_features]
X_test_top = X_test[top_features]

model_top = GradientBoostingRegressor(random_state=42)
model_top.fit(X_train_top, y_train)

y_pred_top = model_top.predict(X_test_top)

mae = mean_absolute_error(y_test, y_pred_top)
mse = mean_squared_error(y_test, y_pred_top)
r2 = r2_score(y_test, y_pred_top)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Top-25 Features Model Performance:")
print(f"MAE:  {mae:.4f}")
print(f"MSE:  {mse:.4f}")
print(f"R2:   {r2:.4f}")
print(f"RMSE: {rmse: .4f}")

In [ ]:
top_features = feat_importance.head(30).index

X_train_top = X_train[top_features]
X_test_top = X_test[top_features]

model_top = GradientBoostingRegressor(random_state=42)
model_top.fit(X_train_top, y_train)

y_pred_top = model_top.predict(X_test_top)

mae = mean_absolute_error(y_test, y_pred_top)
mse = mean_squared_error(y_test, y_pred_top)
r2 = r2_score(y_test, y_pred_top)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Top-30 Features Model Performance:")
print(f"MAE:  {mae:.4f}")
print(f"MSE:  {mse:.4f}")
print(f"R2:   {r2:.4f}")
print(f"RMSE: {rmse: .4f}")

# Trying Grid Search for model improvement

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
}

grid = GridSearchCV(GradientBoostingRegressor(random_state=42), params, cv=3, scoring='neg_root_mean_squared_error')
grid.fit(X_train, y_train)
print("Best RMSE:", -grid.best_score_)
print("Best Params:", grid.best_params_)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("Best Parameters:", grid.best_params_)
print(f"MAE:  {mae:.4f}")
print(f"MSE:  {mse:.4f}")


# Executive Summary: Predicting Incurred Claim Amounts Using Machine Learning

## Objective

The goal of this project was to build a regression model to accurately predict the **incurred claim amount** for insurance cases using structured claim-level features. Accurately forecasting incurred amounts is critical for:

- **Reserving accuracy**
- **Fraud detection**
- **Resource allocation**
- **Premium pricing**

---

## Modeling Approach

We tested multiple algorithms and configurations:
- Models: **Gradient Boosting**, **Random Forest**, and **XGBoost**
- Feature selection: Top 10, 20, 25, and 30 features based on importance
- Hyperparameter tuning using **GridSearchCV**
- Evaluation metrics:
  - **MAE** (Mean Absolute Error)
  - **MSE** (Mean Squared Error)
  - **RMSE** (Root Mean Squared Error)
  - **R²** (Coefficient of Determination)

---

## Key Results

### Model Performance

| Model               | MAE    | MSE     | RMSE   | R²      |
|---------------------|--------|---------|--------|---------|
| Gradient Boosting   | 2.6037 | 9.7264  | 3.1187 | 0.2998  |
| Random Forest       | 2.5873 | 9.9813  | 3.1593 | 0.2814  |
| XGBoost (default)   | 2.6351 | 10.4644 | 3.2349 | 0.2467  |
| XGBoost (GridSearch)| 2.6351 | 10.4644 | **3.0701** | ~     |

### Feature Selection Performance

| Feature Selection   | MAE    | MSE     | RMSE   | R²      |
|---------------------|--------|---------|--------|---------|
| Top-10 Features     | 2.6448 | 9.9464  | 3.2349 | 0.2839  |
| Top-20 Features     | 2.6087 | 9.7660  | 3.2349 | 0.2969  |
| Top-25 Features     | 2.5989 | 9.6996  | 3.2349 | **0.3017**  |
| Top-30 Features     | 2.6047 | 9.7453  | 3.2349 | 0.2984  |

- **Best overall performance**: XGBoost (GridSearchCV) with RMSE = **3.0701**

---

## Business Impact

Accurately predicting incurred amounts provides several key benefits:

1. **Improved Claim Reserving**  
   Better estimation of future liabilities and financial provisioning.

2. **Fraud Detection Support**  
   Discrepancies between predicted and actual incurred values flag potentially suspicious claims.

3. **Underwriting and Pricing Accuracy**  
   Enhances actuarial models and supports risk-based pricing strategies.

4. **Operational Efficiency**  
   Helps prioritize complex or high-value claims automatically.

5. **Scenario Modeling**  
   Allows simulation of financial exposure under different claim distributions or policy changes.

---

## Conclusion

The optimized XGBoost model with tuned hyperparameters yielded the best performance with an RMSE of **3.0701** and R² around **0.30**. This suggests that ~30% of the variance in incurred claim amounts is explained by the features available.

Future improvements can be made by incorporating additional data sources such as:
- NLP-derived insights from claim narratives
- Geographic and behavioral risk factors
- Policyholder history or segmentation

This project lays a strong foundation for **data-driven, scalable, and automated claims management**, contributing to reduced loss ratios, better reserve management, and improved strategic forecasting.

---


# 2. Binary Classification Prediction on High cost Claims

In [ ]:
df['High_Cost_Claim'].unique()
print(df['High_Cost_Claim'].value_counts())
print(df['High_Cost_Claim'].value_counts(normalize=True))
print("Unique classes:", df['High_Cost_Claim'].unique())


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

X = df.drop(columns=['High_Cost_Claim', 'Incurred', 'Capped Incurred', 'Log_Incurred', 'date_of_loss'])
y = df['High_Cost_Claim']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

print("After SMOTE:")
print(y_train.value_counts())

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=5000),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "SVC": SVC(probability=True),
    "Random Forest": RandomForestClassifier(random_state=42)
}

In [ ]:
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)

    results.append({
        'Model': name,
        'Accuracy': acc,
        'Precision': precision,
        'F1 Score': f1,
        'AUC': auc
    })
results_df = pd.DataFrame(results).sort_values(by='AUC', ascending=False)
results_df

# Hyperparameter tuning of our best model Gradient Boosting for improved predictions

In [ ]:
param_grid_gb = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 7],
    "subsample": [0.8, 1.0]
}

In [ ]:
gb_model = GradientBoostingClassifier(random_state=42)

grid_search_gb = GridSearchCV(
    estimator=gb_model,
    param_grid=param_grid_gb,
    scoring='f1',
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_search_gb.fit(X_train, y_train)

In [ ]:
best_gb = grid_search_gb.best_estimator_
y_pred = best_gb.predict(X_test)
y_proba = best_gb.predict_proba(X_test)[:, 1]

print(grid_search_gb.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))

results_df.loc[results_df['Model'] == 'Gradient Boosting', ['Accuracy', 'Precision', 'F1 Score', 'AUC']] = [
    accuracy_score(y_test, y_pred),
    precision_score(y_test, y_pred),
    f1_score(y_test, y_pred),
    roc_auc_score(y_test, y_proba)
]
results_df

In [ ]:
from sklearn.feature_selection import RFE

model = GradientBoostingClassifier(random_state=42)
metrics = []

for k in range(5, X_train.shape[1], 5):
    rfe = RFE(estimator=model, n_features_to_select=k)
    rfe.fit(X_train, y_train)
    
    X_train_rfe = X_train.loc[:, rfe.support_]
    X_test_rfe = X_test.loc[:, rfe.support_]
    
    model.fit(X_train_rfe, y_train)
    y_pred = model.predict(X_test_rfe)
    y_proba = model.predict_proba(X_test_rfe)[:, 1]

    metrics.append({
        "Num Features": k,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_proba)
    })

results_df = pd.DataFrame(metrics)
print(results_df.sort_values(by='F1 Score', ascending=False))


In [ ]:
best_25_features = X_train.columns[rfe.support_][:25]
best_25_features

# Retrain on the top 25 features (final model)

In [ ]:
X_train_top = X_train[best_25_features]
X_test_top = X_test[best_25_features]

In [ ]:
from sklearn.metrics import classification_report

final_model = GradientBoostingClassifier(random_state=42)
final_model.fit(X_train_top, y_train)

y_pred = final_model.predict(X_test_top)
print(classification_report(y_test, y_pred))

## Feature interpretability

In [ ]:
import shap

explainer = shap.Explainer(final_model, X_train_top)
shap_values = explainer(X_test_top)

shap.summary_plot(shap_values, X_test_top, plot_type="bar")


# Report of the Classification Problem

# Executive Summary: Predicting High-Cost Insurance Claims (Classification)

## Objective

The goal of this classification task was to identify **high-cost insurance claims** early in the claim lifecycle using machine learning. Flagging such claims is crucial to proactively manage risk, improve cost control, and optimize claims handling operations.

---

## Modeling Approach

We approached the problem as a **binary classification**:
- **Class 1**: High-cost claims (top 1% based on incurred amount)
- **Class 0**: Normal claims

### Key Steps:
- Used models: **Gradient Boosting**, **XGBoost**, **Random Forest**, **Logistic Regression**, and **Support Vector Classifier (SVC)**
- Conducted **GridSearchCV** hyperparameter tuning
- Applied **Recursive Feature Elimination (RFE)** to optimize feature subset
- Final model retrained with **Top 25 features** (best trade-off)

---

## Model Performance Summary

### Best Performing Model: Gradient Boosting (Top 25 features)

| Metric       | Value     |
|--------------|-----------|
| Accuracy     | **0.9480** |
| Precision    | **0.5100** |
| Recall       | **0.2900** |
| F1 Score     | **0.3700** |
| AUC-ROC      | **0.9054** |

- **Hyperparameters**:  
  `learning_rate = 0.1`, `max_depth = 3`, `n_estimators = 200`, `subsample = 0.8`

### Performance Breakdown (Class 1: High-Cost Claims)

| Metric    | Explanation |
|-----------|-------------|
| **Precision = 0.51** | Out of all predicted high-cost claims, 51% were correct. Acceptable false positive rate. |
| **Recall = 0.29** | Out of actual high-cost claims, only 29% were detected. Needs improvement. |
| **F1 Score = 0.37** | Reflects imbalance between precision and recall. |
| **AUC = 0.9054** | High model separability overall, good signal in data. |

---

## Model Comparison Table (Updated)

| Model               | Accuracy  | Precision | F1 Score | AUC     |
|---------------------|-----------|-----------|----------|---------|
| Logistic Regression | 0.9467    | 0.5294    | 0.3051   | **0.9111**  |
| Gradient Boosting   | 0.9480    | 0.5526    | **0.3443** | 0.9110  |
| Random Forest       | **0.9487**| **0.7273**| 0.1684   | 0.9021  |
| XGBoost             | 0.9448    | 0.4815    | 0.2342   | 0.8691  |
| SVC                 | 0.9454    | 0.0000    | 0.0000   | 0.5445  |

> ✳️ *Gradient Boosting* offers the best **F1 Score** which means a better overall recall, while *Random Forest* achieves the highest **Precision** and **Accuracy**. *Logistic Regression* has the best **AUC**.

---

## 🏁 Feature Selection Performance (RFE Results, 10–35 Features)

| # Features | Accuracy | Precision | F1 Score | AUC     |
|------------|----------|-----------|----------|---------|
| 10         | 0.9461   | 0.5094    | **0.3942** | **0.9198**  |
| 15         | 0.9441   | 0.4783    | 0.3385   | 0.9170  |
| 20         | 0.9454   | 0.5000    | 0.3636   | 0.9128  |
| 25         | 0.9428   | 0.4615    | 0.3529   | 0.9055  |
| 30         | 0.9402   | 0.4259    | 0.3333   | 0.9078  |
| 35         | 0.9441   | 0.4821    | 0.3857   | 0.9088  |

> *10 features* yielded the best **F1 Score** and **AUC**.  
> *20-25 features* were selected as a balance between performance and interpretability.


---

## Business Impact

Predicting high-cost claims enables:

1. **Early Risk Detection (maybe Fraud!)**  
   Flagging costly claims enables proactive case handling, investigations, or negotiation strategies.

2. **Improved Reserves Planning**  
   Enhances financial accuracy by estimating potential large payouts earlier.

3. **Resource Prioritization**  
   Allows fast-track assignment to senior claims handlers or fraud detection teams.

4. **Operational Efficiency**  
   Reduces time-to-resolution and administrative burden on predictable claims.

5. **Strategic Underwriting**  
   Helps inform risk selection and policy design based on learnings from frequent high-cost claim profiles.

---

## Areas for Improvement

- **Low Recall (0.29)** suggests many high-cost claims are still being missed.
- Future steps:
  - **Expand dataset** beyond top 1% cutoff
  - Include **external and unstructured data** (e.g., NLP from claim notes)
  - Fine-tune **classification thresholds** to increase recall while managing precision

---

## Conclusion

The Gradient Boosting Classifier, with tuned parameters and top 20 features, offered the best overall performance. While precision is decent, recall is still low, meaning the model **cautiously predicts** high-cost claims. With further data and modeling iterations, this framework can be refined to **reduce financial exposure** and **improve claims workflow efficiency** across the organization.
